In [2]:
import pandas as pd
import numpy as np

In [52]:
X_test = pd.read_csv('C:/Users/edils/repos/pa004_health_insurance/notebooks/test.csv')

In [53]:
X_test

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,200223,Female,21,1,3.0,1,< 1 Year,No,20408.0,160.0,72
1,49767,Male,55,1,15.0,0,1-2 Year,Yes,37498.0,26.0,102
2,172202,Female,41,1,3.0,0,1-2 Year,Yes,2630.0,26.0,90
3,160714,Female,26,1,11.0,0,< 1 Year,No,2630.0,151.0,268
4,53273,Male,51,1,40.0,0,1-2 Year,Yes,34006.0,124.0,265
...,...,...,...,...,...,...,...,...,...,...,...
76217,258404,Male,23,1,15.0,0,< 1 Year,Yes,23416.0,152.0,172
76218,234156,Female,20,1,15.0,1,< 1 Year,No,32220.0,160.0,233
76219,24477,Female,28,1,8.0,1,< 1 Year,No,49570.0,152.0,205
76220,60424,Female,21,1,3.0,1,< 1 Year,No,32602.0,160.0,54


# Health Insurance Class Render

In [1]:
import pickle
import pandas as pd
import numpy as np
import inflection

class HealthInsurance:
    def __init__(self):
        self.home_path                = 'C:/Users/edils/repos/pa004_health_insurance/health_insurance_app/'
        
        self.age_scaler                           = pickle.load(open(self.home_path + 'features/age_scaler.pkl', 'rb'))
        self.annual_premium_scaler                = pickle.load(open(self.home_path + 'features/annual_premium_scaler.pkl', 'rb'))        
        self.fe_policy_sales_channel_scaler       = pickle.load(open(self.home_path + 'features/fe_policy_sales_channel_scaler.pkl', 'rb'))        
        self.fe_region_code_scaler                = pickle.load(open(self.home_path + 'features/fe_region_code_scaler.pkl', 'rb'))        
        self.vintage_scaler                       = pickle.load(open(self.home_path + 'features/vintage_scaler.pkl', 'rb'))        
        
    def rename_columns(self, df1):
        cols_old = df1.columns

        cols_new = []
        cols_new = cols_old.map(lambda x: inflection.underscore(x))

        df1.columns = cols_new

        return df1
    
    def rename_categorical(self, df2):

        #Renaming vehicle_age lines
        #< 1 Year = new || 1-2 Years = used || >2 Years = old
        df2['vehicle_age'] = df2['vehicle_age'].apply(lambda x: 'old' if x == '> 2 Years' else 'used' if x == '1-2 Year' else 'new')

        #Renaming vehicle_damage
        df2['vehicle_damage'] = df2['vehicle_damage'].apply(lambda x: 1 if x=='Yes' else 0)

        #Renaming gender #Male = 1, Female = 0
        df2['gender'] = df2['gender'].apply(lambda x: 1 if x=='Male' else 0)

        return df2

    def data_encoding(self, df5):

        #vehicle_age - Label Encoder
        df5['vehicle_age'] = df5['vehicle_age'].apply(lambda x: 0 if x == 'new' else 1 if x == 'used' else 2)

        #region_code - Frequency Encoding
        df5['region_code'] = df5['region_code'].map(self.fe_region_code_scaler)

        #policy_sales_channel
        df5['policy_sales_channel'] = df5['policy_sales_channel'].map(self.fe_policy_sales_channel_scaler)    
    
        return df5

    def data_rescalling(self, df5):

        #age 
        df5['age'] = self.age_scaler.transform(df5['age'].values.reshape(-1,1))


        #annual_premium
        df5['annual_premium'] = self.annual_premium_scaler.transform(df5['annual_premium'].values.reshape(-1,1))


        #vintage
        df5['vintage'] = self.vintage_scaler.transform(df5['vintage'].values.reshape(-1,1))
        
        cols_selected = {'vintage',
                         'annual_premium',
                         'age',
                         'region_code',
                         'vehicle_damage',
                         'policy_sales_channel',
                         'previously_insured',
                         'vehicle_age',
                         'gender'}

        return df5[cols_selected]
                  

    def get_prediction(self, model, original_data, test_data):
        
        yhat = model.predict_proba(test_data)
        
        original_data['Prediction'] = yhat[:,1]
        
        return original_data.to_json(orient='records', date_format='iso')

# API Handler Render

In [ ]:
import pickle
import pandas as pd
import os
from flask import Flask, request, Response
from healthinsurance.HealthInsurance import HealthInsurance

#loading model
model = pickle.load(open('models/model_lgb.pkl','rb'))

# initialize API
app = Flask( __name__ )

@app.route( '/healthinsurance/predict', methods=['POST'] )
def healthinsurance_predict():
    test_json = request.get_json()
   
    if test_json: # there is data
        if isinstance( test_json, dict ): # unique example
            test_raw = pd.DataFrame( test_json, index=[0] )
            
        else: # multiple example
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() )
            
        # Instantiate Rossmann class
        pipeline = HealthInsurance()
        
        # data cleaning
        df1 = pipeline.rename_columns( test_raw )
        
        # feature engineering
        df2 = pipeline.rename_categorical( df1 )
        
        # data preparation
        df3 = pipeline.data_encoding( df2 )
        
        df4 = pipeline.data_rescalling(df3)
        
        # prediction
        df_response = pipeline.get_prediction( model, test_raw, df4 )
        
        return df_response
        
        
    else:
        return Response( '{}', status=200, mimetype='application/json' )
    
#Usar isso para testar na maquina
#if __name__ == '__main__':
    #app.run('0.0.0.0', debug=True)

if __name__ == '__main__':
    port = os.environ.get('PORT', 5000)
    app.run(host='0.0.0.0', port=port)

# API Call

In [7]:
import requests
import json

In [60]:
x_test_class = X_test.copy()

In [61]:
x_test_class

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,200223,Female,21,1,3.0,1,< 1 Year,No,20408.0,160.0,72
1,49767,Male,55,1,15.0,0,1-2 Year,Yes,37498.0,26.0,102
2,172202,Female,41,1,3.0,0,1-2 Year,Yes,2630.0,26.0,90
3,160714,Female,26,1,11.0,0,< 1 Year,No,2630.0,151.0,268
4,53273,Male,51,1,40.0,0,1-2 Year,Yes,34006.0,124.0,265
...,...,...,...,...,...,...,...,...,...,...,...
76217,258404,Male,23,1,15.0,0,< 1 Year,Yes,23416.0,152.0,172
76218,234156,Female,20,1,15.0,1,< 1 Year,No,32220.0,160.0,233
76219,24477,Female,28,1,8.0,1,< 1 Year,No,49570.0,152.0,205
76220,60424,Female,21,1,3.0,1,< 1 Year,No,32602.0,160.0,54


In [56]:
x_test_class = json.dumps(x_test_class.to_dict(orient='records'))

In [57]:
#API CALL
#url = 'http://192.168.15.12:5000/healthinsurance/predict'
url = 'https://health-insurance-app-up9u.onrender.com/healthinsurance/predict'
header = {'Content-type' : 'application/json'}

r = requests.post(url, data=x_test_class, headers=header)
print(f'Status Code {r.status_code}')

Status Code 200


In [58]:
df = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [59]:
df

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,Prediction
0,200223,0,0.015385,1,0.024274,1,0,0,-0.750984,0.057146,0.214533,0.010229
1,49767,1,0.538462,1,0.034919,0,1,1,0.388730,0.209127,0.318339,0.066843
2,172202,0,0.323077,1,0.024274,0,1,1,-1.936579,0.209127,0.276817,0.106577
3,160714,0,0.092308,1,0.024224,0,0,0,-1.936579,0.010194,0.892734,0.144240
4,53273,1,0.476923,1,0.003398,0,1,1,0.155852,0.194157,0.882353,0.118014
...,...,...,...,...,...,...,...,...,...,...,...,...
76217,258404,1,0.046154,1,0.034919,0,0,1,-0.550383,0.353663,0.560554,0.089815
76218,234156,0,0.000000,1,0.034919,1,0,0,0.036746,0.057146,0.771626,0.010619
76219,24477,0,0.123077,1,0.088891,1,0,0,1.193798,0.353663,0.674740,0.006670
76220,60424,0,0.015385,1,0.024274,1,0,0,0.062221,0.057146,0.152249,0.005319


In [62]:
path = 'C:/Users/edils/repos/pa004_health_insurance/health_insurance_app/'
model = pickle.load(open(path + '/models/model_lgb.pkl','rb'))

#model = model_lgb_tunned


pipeline = HealthInsurance()

#Data Description
df1 = pipeline.rename_columns(x_test_class)

#Data Transformation
df2 = pipeline.rename_categorical(df1)

#Data Encoding
df3 = pipeline.data_encoding(df2)

#Data Rescalling
df4 = pipeline.data_rescalling(df3)

#prediction
df_response = pipeline.get_prediction(model, x_test_class, df4)

C:\Users\edils\AppData\Local\Temp\ipykernel_15600\2120028006.py:76: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return df5[cols_selected]


In [63]:
pd.read_json(df_response)

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,Prediction
0,200223,0,0.015385,1,0.024274,1,0,0,-0.750984,0.057146,0.214533,0.151943
1,49767,1,0.538462,1,0.034919,0,1,1,0.388730,0.209127,0.318339,0.007036
2,172202,0,0.323077,1,0.024274,0,1,1,-1.936579,0.209127,0.276817,0.009688
3,160714,0,0.092308,1,0.024224,0,0,0,-1.936579,0.010194,0.892734,0.155338
4,53273,1,0.476923,1,0.003398,0,1,1,0.155852,0.194157,0.882353,0.007036
...,...,...,...,...,...,...,...,...,...,...,...,...
76217,258404,1,0.046154,1,0.034919,0,0,1,-0.550383,0.353663,0.560554,0.006783
76218,234156,0,0.000000,1,0.034919,1,0,0,0.036746,0.057146,0.771626,0.098212
76219,24477,0,0.123077,1,0.088891,1,0,0,1.193798,0.353663,0.674740,0.098789
76220,60424,0,0.015385,1,0.024274,1,0,0,0.062221,0.057146,0.152249,0.108475


# Teste Com Render

In [68]:
x_test_class = X_test.copy()

In [69]:
x_test_class = json.dumps(x_test_class.to_dict(orient='records'))

In [70]:
#API CALL
url = 'https://health-insurance-app-up9u.onrender.com/healthinsurance/predict'
header = {'Content-type' : 'application/json'}

r = requests.post(url, data=x_test_class, headers=header)
print(f'Status Code {r.status_code}')

Status Code 200


In [71]:
df_render = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [72]:
df_render.head()

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,Prediction
0,200223,0,0.015385,1,0.024274,1,0,0,-0.750984,0.057146,0.214533,0.010229
1,49767,1,0.538462,1,0.034919,0,1,1,0.388730,0.209127,0.318339,0.066843
2,172202,0,0.323077,1,0.024274,0,1,1,-1.936579,0.209127,0.276817,0.106577
3,160714,0,0.092308,1,0.024224,0,0,0,-1.936579,0.010194,0.892734,0.144240
4,53273,1,0.476923,1,0.003398,0,1,1,0.155852,0.194157,0.882353,0.118014


In [77]:
df_render['Prediction'].sum()

3980.3089242284004

# Teste Local

In [73]:
#API CALL
url = 'http://192.168.15.12:5000/healthinsurance/predict'
header = {'Content-type' : 'application/json'}

r = requests.post(url, data=x_test_class, headers=header)
print(f'Status Code {r.status_code}')

Status Code 200


In [74]:
df_local = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [75]:
df_local.head()

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,Prediction
0,200223,0,0.015385,1,0.024274,1,0,0,-0.750984,0.057146,0.214533,0.032272
1,49767,1,0.538462,1,0.034919,0,1,1,0.388730,0.209127,0.318339,0.005149
2,172202,0,0.323077,1,0.024274,0,1,1,-1.936579,0.209127,0.276817,0.020978
3,160714,0,0.092308,1,0.024224,0,0,0,-1.936579,0.010194,0.892734,0.035023
4,53273,1,0.476923,1,0.003398,0,1,1,0.155852,0.194157,0.882353,0.005542


In [76]:
df_local['Prediction'].sum()

4031.7958088794003